In [370]:
#!pip install psycopg2-binary

In [371]:
#!pip install category_encoders

In [372]:
#!pip install catboost

In [373]:
#!git clone https://github.com/labs15-baseball-pitch-predictor/RC-v1.1--Data-Cleaning-and-Feature-Engineering.git

In [374]:
import pandas as pd
import numpy as np
import random
import pickle
import gc
import psycopg2
#import category_encoders as ce

from sklearn.preprocessing import RobustScaler, StandardScaler
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [375]:
pwd

'/home/ec2-user/SageMaker/RC-v1.1--Data-Cleaning-and-Feature-Engineering/notebooks'

## Load in the pre-pitcher preprocessed_df



In [376]:
#load in pickled preprocessed df
path1 = '/home/ec2-user/SageMaker/RC-v1.1--Data-Cleaning-and-Feature-Engineering/data/pre_pitcher_pre_processed_df_part1.pkl'
path2 = '/home/ec2-user/SageMaker/RC-v1.1--Data-Cleaning-and-Feature-Engineering/data/pre_pitcher_pre_processed_df_part2.pkl'
df1 = pd.read_pickle(path1, compression='zip')
df2 = pd.read_pickle(path2, compression='zip')

df = pd.concat([df1, df2])

In [377]:
#clear some memory:
del df1
del df2
gc.collect()

0

In [378]:
#make list of pitchers (for filling in Nans in batting stat later for pitcher slot in batting order)

def make_pitcher_list(pre_processed_df):
    #make list of all the pitcher ids in the preprocessed_df
    pitcher_list = df['pitcher'].unique().tolist()
    
    #on rare occassions in extra-inning games, a position player is called in to pitch: 
    #to avoid categorizing him as a pitcher we'll only count pitchers with over 100 pitches:
    for pitcher in pitcher_list:
        if len(df[df['pitcher'] == pitcher]) < 100:
            pitcher_list.remove(pitcher)
    return pitcher_list

pitcher_list = make_pitcher_list(df)

## Choose the pitcher to model

In [379]:
#df.groupby('pitcher').count().sort_values('pitch_type', ascending=False).head(10)

In [380]:
#dictionary mapping top 10 pitchers with most pitches name with pitcher id

pitcher_id_dict = {'Justin Verlander': 434378.0, 'Trevor Bauer': 545333.0, 
                   'Aaron Nola': 605400.0, 'Lance Lynn': 458681.0, 'Gerrit Cole':
                   543037.0, 'Jacob deGrom':594798.0, 'Zack Greinke':425844.0,
                   'Patrick Corbin': 571578.0, 'Max Scherzer': 453286.0,
                   'Kyle Gibson': 502043.0}

#make pitcher_df subset where pitcher == picther_id
pitcher_id = pitcher_id_dict['Justin Verlander']
# pitcher_id = pitcher_id_dict['Trevor Bauer']
# pitcher_id = pitcher_id_dict['Aaron Nola']
# pitcher_id = pitcher_id_dict['Lance Lynn']
# pitcher_id = pitcher_id_dict['Gerrit Cole']
# pitcher_id = pitcher_id_dict['Jacob deGrom']
# pitcher_id = pitcher_id_dict['Zack Greinke']
# pitcher_id = pitcher_id_dict['Patrick Corbin']
# pitcher_id = pitcher_id_dict['Max Scherzer']
# pitcher_id = pitcher_id_dict['Kyle Gibson']

pitcher_df = df[df['pitcher'] == pitcher_id]
pitcher_df.head()

,pitch_type,game_date,index,release_speed,release_pos_x,release_pos_z,batter,pitcher,zone,hit_location,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,hc_x,hc_y,fielder_2,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,stand,p_throws,home_team,away_team,type,bb_type,inning_topbot,sv_id,if_fielding_alignment,score_diff,on_1b_id,ball_high,ball_low,ball_left,ball_right,bases_loaded,batter_swung,in_strikezone,chased,fastball_perc_faced,fastball_chase_perc,fastball_bip_swung_perc,fastball_taken_strike_perc,fastball_est_woba,fastball_babip,fastball_iso_value,breaking_perc_faced,breaking_chase_perc,breaking_bip_swung_perc,breaking_taken_strike_perc,breaking_est_woba,breaking_babip,breaking_iso_value,offspeed_perc_faced,offspeed_chase_perc,offspeed_bip_swung_perc,offspeed_taken_strike_perc,offspeed_est_woba,offspeed_babip,offspeed_iso_value,pitchout_perc_faced,balls,strikes,player_name,events,description,des,pitch_name,of_fielding_alignment,_count,count_cat,pitch_cat
1510,FF,2018-03-29,22730,94.599998,-1.9486,6.8034,592261.0,434378.0,8.0,NaN,2018.0,-0.8339,1.6529,0.2748,1.7070,0.0,0.0,0.0,0.0,1.0,NaN,NaN,435263.0,7.4453,-136.895203,-10.8637,-11.885600,32.316799,-9.809500,3.3339,1.4562,NaN,NaN,NaN,92.489998,2652.0,5.498,529411.0,55.000801,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,R,R,TEX,HOU,S,NaN,Bot,180329_195119,Standard,1.0,NaN,False,False,False,False,0,0,1,0,66.241684,21.138212,42.050209,40.000000,0.300159,0.220395,0.098684,26.995565,27.516779,36.702129,43.915344,0.231478,0.200000,0.054545,6.762750,36.231884,21.666666,33.962265,0.426231,0.250000,0.041667,0.000000,0,0,Justin Verlander,NaN,called_strike,NaN,4-Seam Fastball,Standard,00,neutral,fastball
1511,FF,2018-03-29,22711,94.300003,-1.9916,6.6522,592261.0,434378.0,14.0,NaN,2018.0,-1.2892,1.4338,0.4047,1.2020,0.0,0.0,0.0,0.0,1.0,NaN,NaN,435263.0,8.9916,-136.566605,-11.4425,-18.015499,31.964199,-12.041500,3.3446,1.4668,NaN,NaN,NaN,93.283997,2638.0,6.095,529411.0,54.403400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,2.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,R,R,TEX,HOU,B,NaN,Bot,180329_195137,Standard,1.0,NaN,False,True,False,False,0,0,0,0,66.241684,21.138212,42.050209,40.000000,0.300159,0.220395,0.098684,26.995565,27.516779,36.702129,43.915344,0.231478,0.200000,0.054545,6.762750,36.231884,21.666666,33.962265,0.426231,0.250000,0.041667,0.000000,0,1,Justin Verlander,NaN,ball,NaN,4-Seam Fastball,Standard,01,ahead,fastball
1512,SL,2018-03-29,22699,86.400002,-2.1580,6.5734,592261.0,434378.0,14.0,NaN,2018.0,0.2707,0.3526,1.5138,-0.0429,0.0,0.0,0.0,0.0,1.0,NaN,NaN,435263.0,8.0583,-125.145203,-10.0136,1.186800,24.599800,-26.845699,3.3339,1.4562,NaN,NaN,NaN,84.944000,2680.0,5.543,529411.0,54.956001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,3.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,R,R,TEX,HOU,B,NaN,Bot,180329_195201,Standard,1.0,NaN,False,True,False,True,0,0,0,0,66.241684,21.138212,42.050209,40.000000,0.300159,0.220395,0.098684,26.995565,27.516779,36.702129,43.915344,0.231478,0.200000,0.054545,6.762750,36.231884,21.666666,33.962265,0.426231,0.250000,0.041667,0.000000,1,1,Justin Verlander,NaN,ball,NaN,Slider,Standard,11,ahead,breaking
1513,FF,2018-03-29,22691,95.300003,-1.9823,6.7639,592261.0,434378.0,5.0,6.0,2018.0,-1.0804,1.4159,0.1783,2.3724,0.0,0.0,0.0,0.0,1.0,108.040001,149.179993,435263.0,7.9219,-138.185593,-8.8461,-15.272600,30.025200,-12.733900,3.1947,1.2860,145.0,104.199997,7.0,94.040001,2579.0,5.690,529411.0,54.808498,0.674,0.644,0.0,1.0,0.0,0.0,4.0,6.0,4.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,R,R,TEX,HOU,X,line_drive,Bot,180329_195226,Standard,1.0,NaN,False,False,False,False,0,1,1,0,66.241684,21.138212,42.050209,

## Add the pitcher prior tendencies (Pitcher Scouting Report)

In [381]:
%%time
#read in 2017 pre_procesed step 1 pickle and filter for pitcher_id

df_17_path = '/home/ec2-user/SageMaker/RC-v1.1--Data-Cleaning-and-Feature-Engineering/data/preprocessed_2017.pkl'
df17 = pd.read_pickle(df_17_path, compression = 'zip')
pitcher_df17 = df17[df17['pitcher'] == pitcher_id]

CPU times: user 1.82 s, sys: 253 ms, total: 2.07 s
Wall time: 2.07 s


In [382]:
def get_left_right_pitch_tendencies(pitcher_df):
    #split the df into left hand and right handed batters
    left = pitcher_df[pitcher_df['stand'] == 'L'].copy()
    right = pitcher_df[pitcher_df['stand'] == 'R'].copy()
    
    #assign the normalized value counts for this pitchers pitch types to a dictionary
    overall_left = left['pitch_cat'].value_counts(normalize=True).to_dict()
    overall_right = right['pitch_cat'].value_counts(normalize=True).to_dict()
    
    #initialize empty dict for count categories tendencies
    by_count_left = {}
    by_count_right = {}
    
    #loop over each count category and get the pitchers tendencies and add to the dict
    for cat in pitcher_df['count_cat'].unique().tolist():
        left_subset = left[left['count_cat'] == cat]
        right_subset = right[right['count_cat'] == cat]
        
        by_count_left[cat] = left_subset['pitch_cat'].value_counts(normalize=True).to_dict()
        by_count_right[cat] = right_subset['pitch_cat'].value_counts(normalize=True).to_dict()
        
    return overall_left, overall_right, by_count_left, by_count_right

def make_tendency_features(pitcher_df, overall_left, overall_right, by_count_left, by_count_right):
    #helper functions to vectorize w/ df.apply():
    def get_overall_left_perc(x):
        return overall_left[x] * 100
    def get_overall_right_perc(x):
        return overall_right[x] * 100
    def get_by_count_left_perc(x):
        try:
            return by_count_left[x][pitch_type] * 100
        except KeyError:
            return 0
    def get_by_count_right_perc(x):
        try:
            return by_count_right[x][pitch_type] * 100
        except KeyError:
            return 0
    
    left = pitcher_df[pitcher_df['stand'] == 'L'].copy()
    right = pitcher_df[pitcher_df['stand'] == 'R'].copy()
    
    pitch_types_left = overall_left.keys()
    pitch_types_right = overall_right.keys()
    
    #Left
    for pitch_type in pitch_types_left:
        overall_feature = 'overall_' + pitch_type + '_perc'
        count_cat_feature = 'count_cat_' + pitch_type + '_perc'
      
        left[overall_feature] = pitch_type
        left[overall_feature] = left[overall_feature].apply(get_overall_left_perc)
        left[count_cat_feature] = left['count_cat'].apply(get_by_count_left_perc)
    #Right
    for pitch_type in pitch_types_right:
        overall_feature = 'overall_' + pitch_type + '_perc'
        count_cat_feature = 'count_cat_' + pitch_type + '_perc'
      
        right[overall_feature] = pitch_type
        right[overall_feature] = right[overall_feature].apply(get_overall_right_perc)
        right[count_cat_feature] = right['count_cat'].apply(get_by_count_right_perc)
        
    
    return pd.concat([left,right], sort=False).sort_values(by=['game_date', 'game_pk', 'at_bat_number', 'pitch_number'])

def add_pitcher_scouting_report(pitcher_df, pitcher_df17, start_dates, end_dates):
    df = pd.concat([pitcher_df, pitcher_df17], sort=False)
    
    #initialize empty list to store dfs (concat them together later)
    df_list = []
    
    #iterate over each period
    for i in range(len(start_dates)):
      
        #make the prior and current dfs:
        prior_df = df[df['game_date'] < start_dates[i]]
        current_df = df[(df['game_date'] >= start_dates[i]) & (df['game_date'] <= end_dates[i])].copy()
        
        #get the pitch tendencies from prior:
        overall_left, overall_right, by_count_left, by_count_right = get_left_right_pitch_tendencies(prior_df)
        
        #make the pitch tendencies features on current:
        current_df = make_tendency_features(current_df, overall_left, overall_right, by_count_left, by_count_right)
        
        #append the df to the list
        df_list.append(current_df)
    
    df = pd.concat(df_list, sort=False)
    return df

In [383]:
start_dates = ['2018-03-29', '2018-05-01', '2018-06-01', '2018-07-01', '2018-08-01', 
               '2018-09-01', '2019-03-28', '2019-05-01', '2019-06-01', '2019-07-01', 
               '2019-08-01']

end_dates =  ['2018-04-30', '2018-05-31', '2018-06-30', '2018-07-31', '2018-08-31', 
              '2018-10-01', '2019-04-30', '2019-05-31', '2019-06-30', '2019-07-31', 
              '2019-08-31']

In [384]:
%%time
pitcher_df = add_pitcher_scouting_report(pitcher_df, pitcher_df17, start_dates, end_dates)

CPU times: user 2.42 s, sys: 88.3 ms, total: 2.51 s
Wall time: 1.16 s


In [385]:
pitcher_df.head()

,pitch_type,game_date,index,release_speed,release_pos_x,release_pos_z,batter,pitcher,zone,hit_location,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,hc_x,hc_y,fielder_2,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,stand,p_throws,home_team,away_team,type,bb_type,inning_topbot,sv_id,if_fielding_alignment,score_diff,on_1b_id,ball_high,ball_low,ball_left,ball_right,bases_loaded,batter_swung,in_strikezone,chased,fastball_perc_faced,fastball_chase_perc,fastball_bip_swung_perc,fastball_taken_strike_perc,fastball_est_woba,fastball_babip,fastball_iso_value,breaking_perc_faced,breaking_chase_perc,breaking_bip_swung_perc,breaking_taken_strike_perc,breaking_est_woba,breaking_babip,breaking_iso_value,offspeed_perc_faced,offspeed_chase_perc,offspeed_bip_swung_perc,offspeed_taken_strike_perc,offspeed_est_woba,offspeed_babip,offspeed_iso_value,pitchout_perc_faced,balls,strikes,player_name,events,description,des,pitch_name,of_fielding_alignment,_count,count_cat,pitch_cat,overall_fastball_perc,count_cat_fastball_perc,overall_breaking_perc,count_cat_breaking_perc,overall_offspeed_perc,count_cat_offspeed_perc
1510,FF,2018-03-29,22730,94.599998,-1.9486,6.8034,592261.0,434378.0,8.0,NaN,2018.0,-0.8339,1.6529,0.2748,1.7070,0.0,0.0,0.0,0.0,1.0,NaN,NaN,435263.0,7.4453,-136.895203,-10.8637,-11.885600,32.316799,-9.809500,3.3339,1.4562,NaN,NaN,NaN,92.489998,2652.0,5.498,529411.0,55.000801,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,R,R,TEX,HOU,S,NaN,Bot,180329_195119,Standard,1.0,NaN,False,False,False,False,0,0,1,0,66.241684,21.138212,42.050209,40.000000,0.300159,0.220395,0.098684,26.995565,27.516779,36.702129,43.915344,0.231478,0.200000,0.054545,6.762750,36.231884,21.666666,33.962265,0.426231,0.250000,0.041667,0.000000,0,0,Justin Verlander,NaN,called_strike,NaN,4-Seam Fastball,Standard,00,neutral,fastball,59.546127,69.760900,38.973853,30.098453,1.48002,0.140647
1511,FF,2018-03-29,22711,94.300003,-1.9916,6.6522,592261.0,434378.0,14.0,NaN,2018.0,-1.2892,1.4338,0.4047,1.2020,0.0,0.0,0.0,0.0,1.0,NaN,NaN,435263.0,8.9916,-136.566605,-11.4425,-18.015499,31.964199,-12.041500,3.3446,1.4668,NaN,NaN,NaN,93.283997,2638.0,6.095,529411.0,54.403400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,2.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,R,R,TEX,HOU,B,NaN,Bot,180329_195137,Standard,1.0,NaN,False,True,False,False,0,0,0,0,66.241684,21.138212,42.050209,40.000000,0.300159,0.220395,0.098684,26.995565,27.516779,36.702129,43.915344,0.231478,0.200000,0.054545,6.762750,36.231884,21.666666,33.962265,0.426231,0.250000,0.041667,0.000000,0,1,Justin Verlander,NaN,ball,NaN,4-Seam Fastball,Standard,01,ahead,fastball,59.546127,50.609185,38.973853,47.141518,1.48002,2.249297
1512,SL,2018-03-29,22699,86.400002,-2.1580,6.5734,592261.0,434378.0,14.0,NaN,2018.0,0.2707,0.3526,1.5138,-0.0429,0.0,0.0,0.0,0.0,1.0,NaN,NaN,435263.0,8.0583,-125.145203,-10.0136,1.186800,24.599800,-26.845699,3.3339,1.4562,NaN,NaN,NaN,84.944000,2680.0,5.543,529411.0,54.956001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,3.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,R,R,TEX,HOU,B,NaN,Bot,180329_195201,Standard,1.0,NaN,False,True,False,True,0,0,0,0,66.241684,21.138212,42.050209,40.000000,0.300159,0.220395,0.098684,26.995565,27.516779,36.702129,43.915344,0.231478,0.200000,0.054545,6.762750,36.231884,21.666666,33.962265,0.426231,0.250000,0.041667,0.000000,1,1,Justin Verlander,NaN,ball,NaN,Slider,Standard,11,ahead,breaking,59.546127,50.609185,38.973853,47.141518,1.48002,2.249297
1513,FF,2018-03-29,22691,95.300003,-1.9823,6.7639,592261.0,434378.0,5.0,6.0,2018.0,-1.0804,1.4159,0.1783,2.3724,0.0,0.0,0.0,0.0,1.0,108.040001,149.179993,435263.0,7.9219,-138.185593,-8

In [386]:
pitcher_df.description.value_counts()

ball                       1802
foul                       1238
called_strike              1066
swinging_strike             878
hit_into_play               623
hit_into_play_no_out        186
blocked_ball                114
foul_tip                     97
hit_into_play_score          93
swinging_strike_blocked      63
hit_by_pitch                 14
foul_bunt                    14
Name: description, dtype: int64

## Add Game Features:

#### Batting Order, Game pitch_count, & Trailing Pitch Features

In [387]:
pitcher_df.type.head()

1510    S
1511    B
1512    B
1513    X
1514    X
Name: type, dtype: category
Categories (3, object): [B, S, X]

In [388]:
def make_game_batting_order(game_df):
    game_df = game_df.sort_values(by=['at_bat_number', 'pitch_number'])
    all_batters = game_df['batter'].unique().tolist()
    #re-set the at_bat_number for the game to be sequential starting at 1
    at_bat_keys = game_df['at_bat_number'].unique().tolist()
    at_bat_values = range(1, len(at_bat_keys)+1)
    at_bat_map = dict(zip(at_bat_keys, at_bat_values))
    game_df['at_bat_number'] = game_df['at_bat_number'].replace(at_bat_map)
    
    #get the first 9 batter ids
    first_9_batter_subset = game_df[game_df['at_bat_number'] < 10]
    first_9_batters = first_9_batter_subset['batter'].unique().tolist()
    
    #map the batter id to batting order position 1-9
    batting_order_map = dict(zip(first_9_batters, range(1,10)))
    
    #for anyone else who bats later in the game, assign 0 (pinch hitter) to their batting order slot
    other_batters = list(set(all_batters) - set(first_9_batters))
    if len(other_batters) > 0:
        for batter in other_batters:
            batting_order_map[batter] = 0
    try:
        game_df['batting_order_slot'] = game_df['batter'].apply(lambda x: batting_order_map[x])    
    except KeyError:
        game_df = None
        return game_df
    
    game_df['pitcher_AB'] = game_df['batter'].apply(lambda x: 1 if x in pitcher_list else 0)
    game_df['batting_order_slot'] = game_df['batting_order_slot'].where(game_df['pitcher_AB'] == 0, other=10)
    return game_df

def get_pitch_tendencies(pitcher_df):
    #assign the normalized value counts for this pitchers pitch types to a dictionary
    pitcher_tendencies_overall = pitcher_df['pitch_cat'].value_counts(normalize=True).to_dict()

    #initialize empty dict for count categories tendencies
    pitcher_tendencies_by_count = {}
    
    #loop over each count category and get the pitchers tendencies and add to the dict
    for cat in pitcher_df['count_cat'].unique().tolist():
        subset = pitcher_df[pitcher_df['count_cat'] == cat]
        pitcher_tendencies_by_count[cat] = subset['pitch_cat'].value_counts(normalize=True).to_dict()
    return pitcher_tendencies_overall, pitcher_tendencies_by_count

def make_game_pitchcount_and_trailing_pitch_features_and_batting_order(pitcher_df, pitcher_list):
    df = pitcher_df.copy()
    all_games = []
    
    print('#pitches in df before: ' + str(len(df)))
    
    pitcher_tendencies_overall, pitcher_tendencies_by_count = get_pitch_tendencies(df)
    games = df['game_pk'].unique().tolist()
    
    for game in games:
        
        #take the first game and make the pitch count feature
        game_df = df[df['game_pk'] == game].copy()
        game_df['pitch_count'] = range(1, game_df.shape[0] + 1)
    
        #make the L1 pitch features:
        game_df['L1_pitch_type'] = game_df['pitch_type'].shift(periods=1)
        game_df['L1_pitch_result'] = game_df['type'].shift(periods=1)
        game_df['L1_description'] = game_df['description'].shift(periods=1)
        game_df['L1_pitch_zone'] = game_df['zone'].shift(periods=1)
        game_df['L1_batter_swung'] = game_df['batter_swung'].shift(periods=1)
        game_df['L1_chased'] = game_df['chased'].shift(periods=1)        
        
        #add L2 and L3 features:
        
        game_df['L2_pitch_type'] = game_df['pitch_type'].shift(periods=2)
        game_df['L2_pitch_result'] = game_df['type'].shift(periods=2)
        game_df['L2_description'] = game_df['description'].shift(periods=2)
        game_df['L2_pitch_zone'] = game_df['zone'].shift(periods=2)
        game_df['L2_chased'] = game_df['chased'].shift(periods=2)
        
        game_df['L3_pitch_type'] = game_df['pitch_type'].shift(periods=3)
        game_df['L3_description'] = game_df['description'].shift(periods=3)
        game_df['L3_pitch_zone'] = game_df['zone'].shift(periods=3)
        game_df['L3_chased'] = game_df['chased'].shift(periods=3)
        
        str_shifted_cols = ['L1_pitch_type', 'L1_pitch_result','L1_description', 'L2_pitch_type', 
                            'L2_pitch_result','L2_description', 'L3_pitch_type','L3_description']
        
        num_shifted_cols = ['L1_batter_swung', 'L1_chased', 'L1_pitch_zone', 'L2_chased',
                            'L2_pitch_zone', 'L3_chased', 'L3_pitch_zone']
        
        #fill Nan for shifted cols w/ string N/A
        game_df[str_shifted_cols] = game_df[str_shifted_cols].replace({np.nan:'N/A'})
        game_df[num_shifted_cols] = game_df[num_shifted_cols].replace({np.nan:-1})
        
    
        #overall strike % (to fill in for first 5 pitches L5_strike_perc)
        overall_strike_perc = df['type'].value_counts(normalize=True)['S'] * 100

        #make the trailing 5 pitches:
        for index, row in game_df.iterrows():
            #for the first 5 rows, use overall pitcher tendencies    
            if row['pitch_count'] < 6:
                #fill with overall tendencies
                for pitch in list(pitcher_tendencies_overall.keys()):
                    feature = 'L5_' + pitch + '_perc'
                    game_df.at[index, feature] = pitcher_tendencies_overall[pitch] * 100
                    feature = 'L15_' + pitch + '_perc'
                    game_df.at[index, feature] = pitcher_tendencies_overall[pitch] * 100
                #strike %
                game_df.at[index, 'L5_strike_perc'] = overall_strike_perc
                game_df.at[index, 'L15_strike_perc'] = overall_strike_perc

            else:
                current_pitch = game_df.at[index, 'pitch_count']
                #make a subset of the prev 5 pitches
                subset = game_df[(game_df['pitch_count'] > current_pitch - 6) & (game_df['pitch_count'] < current_pitch)]
                #grab the value count percentages for the last 5 pitches
                subset_percentages = subset['pitch_cat'].value_counts(normalize=True).to_dict()
                try:
                    L5_strike_perc = subset['type'].value_counts(normalize=True)['S'] * 100
                except KeyError:
                    L5_strike_perc = 0

                game_df.at[index, 'L5_strike_perc'] = L5_strike_perc

                #iterate over all possible pitch types this pitcher throws:
                for pitch in list(pitcher_tendencies_overall.keys()):
                    feature = 'L5_' + pitch + '_perc'
                    #if he has thrown that pitch type in last 5
                    try:
                        game_df.at[index, feature] = subset_percentages[pitch] * 100
                    #except for when he hasnt thrown that type in last 5
                    except:
                        game_df.at[index, feature] = 0

                if row['pitch_count'] < 16:
                    #make a subset of the prev 15 pitches
                    subset = game_df[(game_df['pitch_count'] < current_pitch)]
                    #grab the value count percentages for the last 15 pitches
                    subset_percentages = subset['pitch_cat'].value_counts(normalize=True).to_dict()
                    try:
                        L15_strike_perc = subset['type'].value_counts(normalize=True)['S'] * 100
                    except KeyError:
                        L15_strike_perc = 0

                    game_df.at[index, 'L15_strike_perc'] = L15_strike_perc

                    #iterate over all possible pitch types this pitcher throws:
                    for pitch in list(pitcher_tendencies_overall.keys()):
                        feature = 'L15_' + pitch + '_perc'
                        #if he has thrown that pitch type in last 15
                        try:
                            game_df.at[index, feature] = subset_percentages[pitch] * 100
                        #except for when he hasnt thrown that type in last 5
                        except:
                            game_df.at[index, feature] = 0
                else:
                    #make a subset of the prev 15 pitches
                    subset = game_df[(game_df['pitch_count'] > current_pitch - 16) & (game_df['pitch_count'] < current_pitch)]
                    #grab the value count percentages for the last 5 pitches
                    subset_percentages = subset['pitch_cat'].value_counts(normalize=True).to_dict()
                    try:
                        L15_strike_perc = subset['type'].value_counts(normalize=True)['S'] * 100
                    except KeyError:
                        L15_strike_perc = 0

                    game_df.at[index, 'L15_strike_perc'] = L15_strike_perc

                    #iterate over all possible pitch types this pitcher throws:
                    for pitch in list(pitcher_tendencies_overall.keys()):
                        feature = 'L15_' + pitch + '_perc'
                        #if he has thrown that pitch type in last 5
                        try:
                            game_df.at[index, feature] = subset_percentages[pitch] * 100
                        #except for when he hasnt thrown that type in last 5
                        except:
                            game_df.at[index, feature] = 0
                            
        #apply the battting order features to the game:                
        game_df = make_game_batting_order(game_df)
    
        all_games.append(game_df)
    
    new_df = pd.concat(all_games).sort_values(by=['game_date', 'game_pk', 'at_bat_number', 'pitch_number'])
    
    print('# pitches in df after: ' + str(len(new_df)))
    
    return new_df

def make_prev_ab_walk_basehit_run_and_homerun_features(pitcher_df):
    
    all_games = []
    #iterate over each game
    for game in pitcher_df['game_pk'].unique():
        #make subset df for that game
        game_df = pitcher_df[pitcher_df['game_pk'] == game].copy()
        #initialize columns to False:
        game_df['prev_ab_run_scored'] = 0
        game_df['prev_ab_homerun'] = 0
        game_df['prev_ab_walk'] = 0
        game_df['prev_ab_basehit'] = 0
        game_df['prev_ab_strikeout'] = 0
        
        #this gets the 
        at_bats = game_df['at_bat_number'].sort_values().unique()
        
        #initialize empty dicts
        run_scored = []
        homeruns = []
        walks = []
        basehits = []
        strikeouts = []
        
        walks = ['walk', 'hit_by_pitch']
        basehits = ['single', 'double', 'triple', 'home_run']
        
        #starting w/ 2nd AB, iterate thru to the end of the at_bats:
        for ab in at_bats[2:]:
            #get the index for the last pitch of the prev AB
            prev_ab_last_pitch_index = game_df[game_df['at_bat_number'] == ab-1]['pitch_number'].index.max()
            #check if the last pitch resulted in a walk or hit by pitch:
            if game_df.loc[prev_ab_last_pitch_index]['events'] in walks:
                #if so, add an entry
                walks.append(ab)
            #check if last pitch gave up a basehit:
            elif game_df.loc[prev_ab_last_pitch_index]['events'] in basehits:
                basehits.append(ab)
            elif game_df.loc[prev_ab_last_pitch_index]['events'] == 'strikeout':
                strikeouts.append(ab)
            
            #to check if prev AB resulted in a run scoring: compare score before and after the AB
            prev_score = game_df[game_df['at_bat_number'] == ab-1]['bat_score'].values[0]
            current_score = game_df[game_df['at_bat_number'] == ab]['bat_score'].values[0]
            
            if current_score > prev_score:
                run_scored.append(ab)
                 
                #check if last AB gave up a homerun:
                if game_df.loc[prev_ab_last_pitch_index]['events'] == 'home_run':
                    homeruns.append(ab)
                    
        #iterate over each at_bat, and add the features to the df where appropriate
        for ab in at_bats:
            idx = game_df[game_df['at_bat_number'] == ab].index
            if ab in walks:
                game_df.at[idx, 'prev_ab_walk'] = 1
            elif ab in basehits:
                game_df.at[idx, 'prev_ab_basehit'] = 1
            elif ab in strikeouts:
                game_df.at[idx, 'prev_ab_strikeout'] = 1
            if ab in run_scored:
                game_df.at[idx, 'prev_ab_run_scored'] = 1
                if ab in homeruns:
                    game_df.at[idx, 'prev_ab_homerun'] = 1
        all_games.append(game_df)
        
    return pd.concat(all_games).sort_values(by=['game_date', 'game_pk', 'pitch_count'])

In [389]:
%%time
pitcher_df = make_game_pitchcount_and_trailing_pitch_features_and_batting_order(pitcher_df, pitcher_list)

#pitches in df before: 6188
# pitches in df after: 6188
CPU times: user 1min 8s, sys: 5.81 ms, total: 1min 8s
Wall time: 1min 8s


In [390]:
%%time
pitcher_df = make_prev_ab_walk_basehit_run_and_homerun_features(pitcher_df)

CPU times: user 17.7 s, sys: 0 ns, total: 17.7 s
Wall time: 17.7 s


In [391]:
pitcher_df[['description', 'type', 'events','des', 'batter_swung', 'chased']].head(10)

,description,type,events,des,batter_swung,chased
1510,called_strike,S,NaN,NaN,0,0
1511,ball,B,NaN,NaN,0,0
1512,ball,B,NaN,NaN,0,0
1513,hit_into_play,X,field_out,Delino DeShields lines out to shortstop Carlos...,1,0
1514,hit_into_play,X,field_out,Joey Gallo pops out to third baseman Alex Breg...,1,1
1515,called_strike,S,NaN,NaN,0,0
1516,hit_into_play,X,field_out,Elvis Andrus flies out to left fielder Josh Re...,1,1
1531,called_strike,S,NaN,NaN,0,0
1532,called_strike,S,NaN,NaN,0,0
1533,ball,B,NaN,NaN,0,0


In [392]:
cols = ['pitch_count', 'pitch_type', 'type', 'batting_order_slot']

for col in pitcher_df.columns:
    if col[:1] == 'L':
        cols.append(col)
    if col[-4:] == 'perc':
        if col[:2]=='L5':
            cols.append(col)
        if col[:3]=='L15':
            cols.append(col)
        
pitcher_df[cols].head(20)

,pitch_count,pitch_type,type,batting_order_slot,L1_pitch_type,L1_pitch_result,L1_description,L1_pitch_zone,L1_batter_swung,L1_chased,L2_pitch_type,L2_pitch_result,L2_description,L2_pitch_zone,L2_chased,L3_pitch_type,L3_description,L3_pitch_zone,L3_chased,L5_fastball_perc,L5_fastball_perc,L15_fastball_perc,L15_fastball_perc,L5_breaking_perc,L5_breaking_perc,L15_breaking_perc,L15_breaking_perc,L5_offspeed_perc,L5_offspeed_perc,L15_offspeed_perc,L15_offspeed_perc,L5_strike_perc,L5_strike_perc,L15_strike_perc,L15_strike_perc
1510,1,FF,S,1,N/A,N/A,N/A,-1.0,-1.0,-1.0,N/A,N/A,N/A,-1.0,-1.0,N/A,N/A,-1.0,-1.0,56.415643,56.415643,56.415643,56.415643,40.982547,40.982547,40.982547,40.982547,2.60181,2.60181,2.60181,2.60181,54.250162,54.250162,54.250162,54.250162
1511,2,FF,B,1,FF,S,called_strike,8.0,0.0,0.0,N/A,N/A,N/A,-1.0,-1.0,N/A,N/A,-1.0,-1.0,56.415643,56.415643,56.415643,56.415643,40.982547,40.982547,40.982547,40.982547,2.60181,2.60181,2.60181,2.60181,54.250162,54.250162,54.250162,54.250162
1512,3,SL,B,1,FF,B,ball,14.0,0.0,0.0,FF,S,called_strike,8.0,0.0,N/A,N/A,-1.0,-1.0,56.415643,56.415643,56.415643,56.415643,40.982547,40.982547,40.982547,40.982547,2.60181,2.60181,2.60181,2.60181,54.250162,54.250162,54.250162,54.250162
1513,4,FF,X,1,SL,B,ball,14.0,0.0,0.0,FF,B,ball,14.0,0.0,FF,called_strike,8.0,0.0,56.415643,56.415643,56.415643,56.415643,40.982547,40.982547,40.982547,40.982547,2.60181,2.60181,2.60181,2.60181,54.250162,54.250162,54.250162,54.250162
1514,5,FF,X,2,FF,X,hit_into_play,5.0,1.0,0.0,SL,B,ball,14.0,0.0,FF,ball,14.0,0.0,56.415643,56.415643,56.415643,56.415643,40.982547,40.982547,40.982547,40.982547,2.60181,2.60181,2.60181,2.60181,54.250162,54.250162,54.250162,54.250162
1515,6,FF,S,3,FF,X,hit_into_play,1.0,1.0,1.0,FF,X,hit_into_play,5.0,0.0,SL,ball,14.0,0.0,80.000000,80.000000,80.000000,80.000000,20.000000,20.000000,20.000000,20.000000,0.00000,0.00000,0.00000,0.00000,20.000000,20.000000,20.000000,20.000000
1516,7,FF,X,3,FF,S,called_strike,3.0,0.0,0.0,FF,X,hit_into_play,1.0,1.0,FF,hit_into_play,5.0,0.0,80.000000,80.000000,83.333333,83.333333,20.000000,20.000000,16.666667,16.666667,0.00000,0.00000,0.00000,0.00000,20.000000,20.000000,33.333333,33.333333
1531,8,FF,S,4,FF,X,hit_into_play,4.0,1.0,1.0,FF,S,called_strike,3.0,0.0,FF,hit_into_play,1.0,1.0,80.000000,80.000000,85.714286,85.714286,20.000000,20.000000,14.285714,14.285714,0.00000,0.00000,0.00000,0.00000,20.000000,20.000000,28.571429,28.571429
1532,9,CU,S,4,FF,S,called_strike,8.0,0.0,0.0,FF,X,hit_into_play,4.0,1.0,FF,called_strike,3.0,0.0,100.000000,100.000000,87.500000,87.500000,0.000000,0.000000,12.500000,12.500000,0.00000,0.00000,0.00000,0.00000,40.000000,40.000000,37.500000,37.500000
1533,10,SL,B,4,CU,S,called_strike,6.0,0.0,0.0,FF,S,called_strike,8.0,0.0,FF,hit_into_play,4.0,1.0,80.000000,80.000000,77.777778,77.777778,20.000000,20.000000,22.222222,22.222222,0.00000,0.00000,0.00000,0.00000,60.000000,60.000000,44.444444,44.444444


In [393]:
#pitcher_df[cols].tail(20)

## Fill Batting Scouting Report NaNs

#### Load in the batting_order_slot map pickle

In [394]:
batting_path = '/home/ec2-user/SageMaker/RC-v1.1--Data-Cleaning-and-Feature-Engineering/data/batting_order_slot_map.pkl'
batting_order_slot_map = pickle.load(open(batting_path, "rb"))

In [395]:
batting_order_slot_map[10] = batting_order_slot_map['pitcher']
batting_order_slot_map[0] = batting_order_slot_map['PH']

In [396]:
batting_order_slot_map[0]

{'fastball_perc_faced': 61.927233894556956,
 'fastball_chase_perc': 29.40695728934851,
 'fastball_bip_swung_perc': 37.81120738654654,
 'fastball_taken_strike_perc': 29.976635717520725,
 'fastball_est_woba': 0.369995114195133,
 'fastball_babip': 0.1994954393784011,
 'fastball_iso_value': 0.14904537669087212,
 'breaking_perc_faced': 27.28474743523461,
 'breaking_chase_perc': 32.83739055354618,
 'breaking_bip_swung_perc': 33.79759539794321,
 'breaking_taken_strike_perc': 35.8911963136291,
 'breaking_est_woba': 0.3436877788414853,
 'breaking_babip': 0.1669350920699022,
 'breaking_iso_value': 0.1287283785928761,
 'offspeed_perc_faced': 10.797045853944619,
 'offspeed_chase_perc': 37.74180972842209,
 'offspeed_bip_swung_perc': 38.43831885840335,
 'offspeed_taken_strike_perc': 21.321336439373177,
 'offspeed_est_woba': 0.3385999366000445,
 'offspeed_babip': 0.18845201882223023,
 'offspeed_iso_value': 0.13874368800766018,
 'pitchout_perc_faced': 0.009129285663704204}

In [397]:
batter_cols = ['fastball_perc_faced','fastball_chase_perc','fastball_bip_swung_perc', 'fastball_taken_strike_perc',
               'fastball_est_woba', 'fastball_babip', 'fastball_iso_value', 'breaking_perc_faced', 'breaking_chase_perc',
               'breaking_bip_swung_perc', 'breaking_taken_strike_perc', 'breaking_est_woba', 'breaking_babip', 
               'breaking_iso_value', 'offspeed_perc_faced', 'offspeed_chase_perc', 'offspeed_bip_swung_perc',
               'offspeed_taken_strike_perc', 'offspeed_est_woba', 'offspeed_babip', 'offspeed_iso_value',
               'pitchout_perc_faced']

def fill_batting_nans(pitcher_df, batting_order_slot_map):
    df = pitcher_df.copy()
    for slot in df['batting_order_slot'].unique().tolist():
        subset = df[df['batting_order_slot'] == slot].copy()
        df = df.drop(subset.index)
        for col in batter_cols:
            subset[col] = subset[col].fillna(batting_order_slot_map[slot][col])
        df = pd.concat([df, subset])
        print('finished w/ slot: ' + str(slot))
    df = df.sort_values(by=['game_date', 'game_pk', 'pitch_count'])
    return df

In [398]:
%%time
pitcher_df = fill_batting_nans(pitcher_df, batting_order_slot_map)

finished w/ slot: 1
finished w/ slot: 2
finished w/ slot: 3
finished w/ slot: 4
finished w/ slot: 5
finished w/ slot: 6
finished w/ slot: 7
finished w/ slot: 8
finished w/ slot: 9
finished w/ slot: 10
finished w/ slot: 0
CPU times: user 467 ms, sys: 8.13 ms, total: 475 ms
Wall time: 468 ms


## Add Pitcher / Batter Matchup History Report

In [399]:
def add_pb_matchup_priors(pitcher_df, pitcher_df17, start_dates, end_dates):
    df = pd.concat([pitcher_df, pitcher_df17], sort=False)
    
    #initialize empty list to store dfs (concat them together later)
    df_list = []
    
    #iterate over each period
    for i in range(len(start_dates)):
        
        #make the prior and current dfs:
        prior_df = df[df['game_date'] < start_dates[i]]
        current_df = df[(df['game_date'] >= start_dates[i]) & (df['game_date'] <= end_dates[i])]
        
        #get all the pitch_types this pitcher has thrown in the past:
        pitch_types = prior_df['pitch_cat'].unique().tolist()
        
        try:
            pitch_types.remove('PO')
        except:
            pass
        
        #get a list of the batters in the current_df
        current_batters = current_df['batter'].unique().tolist()
        
        batters_dict = {}
        
        current_df_list = []
        
        for batter in current_batters:
            batter_df_list = []
            
            #first use subset from prior df
            batter_subset = prior_df[prior_df['batter'] == batter].copy()
            #if pitcher has never faced this batter before:
            if batter_subset.empty:
                #get the left or right handedness of the batter
                stand = current_df[current_df['batter'] == batter]['stand'].values[0]
                #use overall prior tendencies vs left or right handed hitters
                overall, by_count = get_pitch_tendencies(prior_df[prior_df['stand'] == stand])
            else:
                overall, by_count = get_pitch_tendencies(batter_subset)
            batters_dict[batter] = by_count
            
            #now use subset of current_df where batter=batter
            batter_subset = current_df[current_df['batter'] == batter].copy()
            #iterate over the different count_cat types:
            for count_cat in ['ahead', 'behind', 'neutral']:
                count_subset = batter_subset[batter_subset['count_cat'] == count_cat].copy()
                if count_subset.empty:
                    continue
                else:
                    for pitch in pitch_types:
                        try:
                            count_subset['PB_'+pitch] = batters_dict[batter][count_cat][pitch] * 100
                        except KeyError:
                            count_subset['PB_'+pitch] = 0
                            
                current_df_list.append(count_subset)
        
        current_df = pd.concat(current_df_list, sort=False)
        df_list.append(current_df)
                    
    new_df = pd.concat(df_list, sort=False).sort_values(by=['game_date', 'game_pk', 'pitch_count'])
    return new_df

In [400]:
%%time
pitcher_df = add_pb_matchup_priors(pitcher_df, pitcher_df17, start_dates, end_dates)

CPU times: user 26.9 s, sys: 76.3 ms, total: 27 s
Wall time: 25.6 s


In [401]:
pitcher_df17.pitch_type.value_counts()

FF    2371
SL     876
CU     614
CH     161
FC      31
FT      12
Name: pitch_type, dtype: int64

In [402]:
pitcher_df.pitch_type.value_counts()

FF    3470
SL    1527
CU    1009
CH     161
FC      21
Name: pitch_type, dtype: int64

In [403]:
pitcher_df17.pitch_cat.value_counts(normalize=True)

fastball    0.593850
breaking    0.366544
offspeed    0.039606
Name: pitch_cat, dtype: float64

In [404]:
pitcher_df.pitch_cat.value_counts(normalize=True)

fastball    0.564156
breaking    0.409825
offspeed    0.026018
Name: pitch_cat, dtype: float64

In [405]:
pitcher_df.head(1)

,pitch_type,game_date,index,release_speed,release_pos_x,release_pos_z,batter,pitcher,zone,hit_location,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,hc_x,hc_y,fielder_2,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,stand,p_throws,home_team,away_team,type,bb_type,inning_topbot,sv_id,if_fielding_alignment,score_diff,on_1b_id,ball_high,ball_low,ball_left,ball_right,bases_loaded,batter_swung,in_strikezone,chased,fastball_perc_faced,fastball_chase_perc,fastball_bip_swung_perc,fastball_taken_strike_perc,fastball_est_woba,fastball_babip,fastball_iso_value,breaking_perc_faced,breaking_chase_perc,breaking_bip_swung_perc,breaking_taken_strike_perc,breaking_est_woba,breaking_babip,breaking_iso_value,offspeed_perc_faced,offspeed_chase_perc,offspeed_bip_swung_perc,offspeed_taken_strike_perc,offspeed_est_woba,offspeed_babip,offspeed_iso_value,pitchout_perc_faced,balls,strikes,player_name,events,description,des,pitch_name,of_fielding_alignment,_count,count_cat,pitch_cat,overall_fastball_perc,count_cat_fastball_perc,overall_breaking_perc,count_cat_breaking_perc,overall_offspeed_perc,count_cat_offspeed_perc,pitch_count,L1_pitch_type,L1_pitch_result,L1_description,L1_pitch_zone,L1_batter_swung,L1_chased,L2_pitch_type,L2_pitch_result,L2_description,L2_pitch_zone,L2_chased,L3_pitch_type,L3_description,L3_pitch_zone,L3_chased,L5_fastball_perc,L15_fastball_perc,L5_breaking_perc,L15_breaking_perc,L5_offspeed_perc,L15_offspeed_perc,L5_strike_perc,L15_strike_perc,batting_order_slot,pitcher_AB,prev_ab_run_scored,prev_ab_homerun,prev_ab_walk,prev_ab_basehit,prev_ab_strikeout,PB_fastball,PB_breaking,PB_offspeed
1510,FF,2018-03-29,22730,94.599998,-1.9486,6.8034,592261.0,434378.0,8.0,NaN,2018.0,-0.8339,1.6529,0.2748,1.707,0.0,0.0,0.0,0.0,1.0,NaN,NaN,435263.0,7.4453,-136.895203,-10.8637,-11.8856,32.316799,-9.8095,3.3339,1.4562,NaN,NaN,NaN,92.489998,2652.0,5.498,529411.0,55.000801,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,R,R,TEX,HOU,S,NaN,Bot,180329_195119,Standard,1.0,NaN,False,False,False,False,0,0,1,0,66.241684,21.138212,42.050209,40.0,0.300159,0.220395,0.098684,26.995565,27.516779,36.702129,43.915344,0.231478,0.2,0.054545,6.76275,36.231884,21.666666,33.962265,0.426231,0.25,0.041667,0.0,0,0,Justin Verlander,NaN,called_strike,NaN,4-Seam Fastball,Standard,00,neutral,fastball,59.546127,69.7609,38.973853,30.098453,1.48002,0.140647,1.0,N/A,N/A,N/A,-1.0,-1.0,-1.0,N/A,N/A,N/A,-1.0,-1.0,N/A,N/A,-1.0,-1.0,56.415643,56.415643,40.982547,40.982547,2.60181,2.60181,54.250162,54.250162,1.0,0.0,0.0,0.0,0.0,0.0,0.0,71.428571,28.571429,0.0


In [406]:
#pitcher_df[pitcher_df['release_speed'].isna()][['release_speed', 'pitch_type','pitch_name']]

## Choose features to use in model input vector, re-cast datatypes, encode categorical variables (onehot and binary), scale appropriately (Standard vs MinMax vs Robust), and check for any remaining NaNs and choose imputation strategy

#### Train/test Split (IMO use April/2017 - end of June/2019 as Train, and July/August 2019 as Test)

In [407]:
# model_features = ['game_date', 'batter', 'on_3b', 'on_2b', 'on_1b', 'outs_when_up',
#                 'inning','fielder_2', 'sz_top', 'sz_bot', 'game_pk','at_bat_number',
#                 'pitch_number', 'bat_score', 'fld_score', 'stand', 'p_throws',
#                 'inning_topbot', 'if_fielding_alignment','score_diff', 'on_1b_id',
#                 #'L1_ball_high', 'L1_ball_low', 'L1_ball_left', 'L1_ball_right',
#                 'bases_loaded', 'fastball_perc_faced', 'fastball_chase_perc',
#                 'fastball_bip_swung_perc', 'fastball_taken_strike_perc',
#                 'fastball_est_woba', 'fastball_babip', 'fastball_iso_value',
#                 'breaking_perc_faced', 'breaking_chase_perc', 'breaking_bip_swung_perc',
#                 'breaking_taken_strike_perc', 'breaking_est_woba', 'breaking_babip',
#                 'breaking_iso_value', 'offspeed_perc_faced', 'offspeed_chase_perc',
#                 'offspeed_bip_swung_perc', 'offspeed_taken_strike_perc',
#                 'offspeed_est_woba', 'offspeed_babip', 'offspeed_iso_value',
#                 'pitchout_perc_faced', 'balls','strikes', 'of_fielding_alignment',
#                 '_count', 'count_cat', 'overall_breaking_perc',
#                 'count_cat_breaking_perc', 'overall_fastball_perc',
#                 'count_cat_fastball_perc', 'overall_offspeed_perc',
#                 'count_cat_offspeed_perc', 'pitch_count', 'L1_pitch_type',
#                 'L1_pitch_result', 'L1_pitch_zone', 'L2_pitch_type',
#                 'L2_description', 'L2_pitch_zone','L3_pitch_type',
#                 'L3_description', 'L3_pitch_zone', 'L5_fastball_perc', 
#                 'L15_fastball_perc', 'L5_breaking_perc', 'L15_breaking_perc',
#                 'L5_offspeed_perc', 'L15_offspeed_perc', 'L5_strike_perc',
#                 'L15_strike_perc', 'batting_order_slot', 'pitcher_AB', 
#                 'prev_ab_run_scored', 'prev_ab_homerun', 'prev_ab_walk',
#                 'prev_ab_basehit', 'prev_ab_strikeout', 'PB_fastball', 'PB_breaking',
#                 'PB_offspeed', 'pitch_cat']



In [408]:
drop_cols = ['index','release_speed','release_pos_x','release_pos_z','hit_location',
             'game_year', 'pfx_x', 'pfx_z', 'plate_x', 'plate_z', 'hc_x', 'hc_y', 'vx0',
             'vy0', 'vz0', 'ax', 'ay', 'az','hit_distance_sc', 'launch_speed', 'launch_angle',
             'effective_speed', 'release_spin_rate', 'release_extension', 'release_pos_y',
             'estimated_ba_using_speedangle', 'estimated_woba_using_speedangle', 'woba_value',
             'woba_denom', 'babip_value', 'iso_value', 'launch_speed_angle', 'home_score',
             'away_score', 'fld_score', 'post_bat_score', 'post_fld_score', 'p_throws',
             'home_team', 'away_team', 'type', 'bb_type', 'sv_id', 'ball_high', 'ball_low',
             'ball_left', 'ball_right', 'batter_swung', 'in_strikezone', 'chased','events',
             'description', 'des', 'pitch_name', 'zone', 'post_away_score', 'post_home_score',
             'batter', 'on_1b_id', 'game_pk']

In [409]:
def extract_date_features(df):
    df = df.copy()
    
    #df['day'] = df['game_date'].dt.day
    df['month'] = df['game_date'].dt.month
    df['year'] = df['game_date'].dt.year
    
    df = df.sort_values(by=['game_date', 'game_pk', 'pitch_count'])
    #df = df.drop('game_date', axis=1)
    return df

#convert datetime into just month and year
model_df = extract_date_features(pitcher_df)

#drop columns before pickling
model_df = model_df.drop(columns=drop_cols)

#rename fielder_2 to catcher_id:
model_df = model_df.rename(columns={'fielder_2':'catcher_id'})

#add bool for whether there are ANY runners on base:
model_df['on_base'] = ((model_df['on_3b'] + model_df['on_2b'] + model_df['on_1b']) > 0) * 1

#fillna for on_1b_id:
#model_df['on_1b_id'] = model_df['on_1b_id'].fillna('N/A')

#fillna for infield and outfield alignment:
model_df[['if_fielding_alignment', 'of_fielding_alignment']] = model_df[['if_fielding_alignment', 'of_fielding_alignment']].fillna('Standard')

#fillna for sz_top and sz_bot
model_df['sz_top'] = model_df['sz_top'].fillna(model_df['sz_top'].median())
model_df['sz_bot'] = model_df['sz_bot'].fillna(model_df['sz_bot'].median())

#fillna for catcher_id w/ most frequent catcher for that pitcher:
catcher = model_df['catcher_id'].value_counts().index[0]
model_df['catcher_id'] = model_df['catcher_id'].fillna(catcher)

#convert cols to categorical:
cat_cols = ['pitch_type', 'balls', 'strikes', 'player_name', 'of_fielding_alignment', '_count', 'count_cat', 'pitch_cat',
              'L1_pitch_type', 'L1_pitch_result', 'L1_description', 'L1_pitch_zone', 'L1_batter_swung', 'L1_chased', 'L2_pitch_type',
              'L2_pitch_result', 'L2_description', 'L2_pitch_zone', 'L2_chased', 'L3_pitch_type', 'L3_description', 'L3_pitch_zone',
              'L3_chased', 'batting_order_slot', 'pitcher', 'outs_when_up', 'inning', 'catcher_id', 'year', 'month',
              'at_bat_number', 'pitch_number', 'pitch_count']
model_df[cat_cols] = model_df[cat_cols].astype('category')


In [410]:
model_df.head()

,pitch_type,game_date,pitcher,on_3b,on_2b,on_1b,outs_when_up,inning,catcher_id,sz_top,sz_bot,at_bat_number,pitch_number,bat_score,stand,inning_topbot,if_fielding_alignment,score_diff,bases_loaded,fastball_perc_faced,fastball_chase_perc,fastball_bip_swung_perc,fastball_taken_strike_perc,fastball_est_woba,fastball_babip,fastball_iso_value,breaking_perc_faced,breaking_chase_perc,breaking_bip_swung_perc,breaking_taken_strike_perc,breaking_est_woba,breaking_babip,breaking_iso_value,offspeed_perc_faced,offspeed_chase_perc,offspeed_bip_swung_perc,offspeed_taken_strike_perc,offspeed_est_woba,offspeed_babip,offspeed_iso_value,pitchout_perc_faced,balls,strikes,player_name,of_fielding_alignment,_count,count_cat,pitch_cat,overall_fastball_perc,count_cat_fastball_perc,overall_breaking_perc,count_cat_breaking_perc,overall_offspeed_perc,count_cat_offspeed_perc,pitch_count,L1_pitch_type,L1_pitch_result,L1_description,L1_pitch_zone,L1_batter_swung,L1_chased,L2_pitch_type,L2_pitch_result,L2_description,L2_pitch_zone,L2_chased,L3_pitch_type,L3_description,L3_pitch_zone,L3_chased,L5_fastball_perc,L15_fastball_perc,L5_breaking_perc,L15_breaking_perc,L5_offspeed_perc,L15_offspeed_perc,L5_strike_perc,L15_strike_perc,batting_order_slot,pitcher_AB,prev_ab_run_scored,prev_ab_homerun,prev_ab_walk,prev_ab_basehit,prev_ab_strikeout,PB_fastball,PB_breaking,PB_offspeed,month,year,on_base
1510,FF,2018-03-29,434378.0,0.0,0.0,0.0,0.0,1.0,435263.0,3.3339,1.4562,1.0,1.0,0.0,R,Bot,Standard,1.0,0,66.241684,21.138212,42.050209,40.000000,0.300159,0.220395,0.098684,26.995565,27.516779,36.702129,43.915344,0.231478,0.200000,0.054545,6.762750,36.231884,21.666666,33.962265,0.426231,0.250000,0.041667,0.000000,0,0,Justin Verlander,Standard,00,neutral,fastball,59.546127,69.760900,38.973853,30.098453,1.48002,0.140647,1.0,N/A,N/A,N/A,-1.0,-1.0,-1.0,N/A,N/A,N/A,-1.0,-1.0,N/A,N/A,-1.0,-1.0,56.415643,56.415643,40.982547,40.982547,2.60181,2.60181,54.250162,54.250162,1.0,0.0,0.0,0.0,0.0,0.0,0.0,71.428571,28.571429,0.000000,3,2018,0
1511,FF,2018-03-29,434378.0,0.0,0.0,0.0,0.0,1.0,435263.0,3.3446,1.4668,1.0,2.0,0.0,R,Bot,Standard,1.0,0,66.241684,21.138212,42.050209,40.000000,0.300159,0.220395,0.098684,26.995565,27.516779,36.702129,43.915344,0.231478,0.200000,0.054545,6.762750,36.231884,21.666666,33.962265,0.426231,0.250000,0.041667,0.000000,0,1,Justin Verlander,Standard,01,ahead,fastball,59.546127,50.609185,38.973853,47.141518,1.48002,2.249297,2.0,FF,S,called_strike,8.0,0.0,0.0,N/A,N/A,N/A,-1.0,-1.0,N/A,N/A,-1.0,-1.0,56.415643,56.415643,40.982547,40.982547,2.60181,2.60181,54.250162,54.250162,1.0,0.0,0.0,0.0,0.0,0.0,0.0,27.777778,72.222222,0.000000,3,2018,0
1512,SL,2018-03-29,434378.0,0.0,0.0,0.0,0.0,1.0,435263.0,3.3339,1.4562,1.0,3.0,0.0,R,Bot,Standard,1.0,0,66.241684,21.138212,42.050209,40.000000,0.300159,0.220395,0.098684,26.995565,27.516779,36.702129,43.915344,0.231478,0.200000,0.054545,6.762750,36.231884,21.666666,33.962265,0.426231,0.250000,0.041667,0.000000,1,1,Justin Verlander,Standard,11,ahead,breaking,59.546127,50.609185,38.973853,47.141518,1.48002,2.249297,3.0,FF,B,ball,14.0,0.0,0.0,FF,S,called_strike,8.0,0.0,N/A,N/A,-1.0,-1.0,56.415643,56.415643,40.982547,40.982547,2.60181,2.60181,54.250162,54.250162,1.0,0.0,0.0,0.0,0.0,0.0,0.0,27.777778,72.222222,0.000000,3,2018,0
1513,FF,2018-03-29,434378.0,0.0,0.0,0.0,0.0,1.0,435263.0,3.1947,1.2860,1.0,4.0,0.0,R,Bot,Standard,1.0,0,66.241684,21.138212,42.050209,40.000000,0.300159,0.220395,0.098684,26.995565,27.516779,36.702129,43.915344,0.231478,0.200000,0.054545,6.762750,36.231884,21.666666,33.962265,0.426231,0.250000,0.041667,0.000000,2,1,Justin Verlander,Standard,21,neutral,fastball,59.546127,69.760900,38.973853,30.098453,1.48002,0.140647,4.0,SL,B,ball,14.0,0.0,0.0,FF,B,ball,14.0,0.0,FF,called_strike,8.0,0.0,56.415643,56.415643,40.982547,40.982547,2.60181,2.60181,54.250162,54.250162,1.0,0.0,0.0,0.0,0.0,0.0,0.0,71.428571,28.571429,0.000000,3,2018,0
1514,FF,2018-03-29,434378.0,0.0,0.0,0.0,1.0,1.0,435263.0,3.4964,1.5073,2.0,1.0,0.0,L,Bo

In [411]:
#model_df.isna().sum()

In [412]:
#model_df.info()

In [413]:
# import category_encoders as ce
# from sklearn.preprocessing import RobustScaler, StandardScaler

# #categoricals cols:


# #one hot encode categorical columns:
# one_hot_cols = ['outs_when_up', 'stand', 'inning_topbot', 'if_fielding_alignment',
#                 'of_fielding_alignment', 'count_cat', 'L1_pitch_type', 
#                 'L1_pitch_result', 'L1_ball_high', 'L1_ball_low', 'L1_ball_left', 
#                 'L1_ball_right','year', 'p_throws', 'L2_pitch_type',
#                 'L2_description', 'L2_pitch_zone','L3_pitch_type',
#                 'L3_description', 'L3_pitch_zone']

# #binary encode categorical columns:
# binary_cols = ['inning', 'batter', 'catcher_id', 'game_pk', 'at_bat_number', 'pitch_number',
#                'bat_score', 'fld_score', 'score_diff', 'on_1b_id', 'balls', 'strikes', 
#                '_count', 'pitch_count', 'L1_pitch_zone', 'batting_order_slot', 
#                'month']
    

# #robust scaler cols:

# numeric_cols = ['fastball_perc_faced', 'fastball_chase_perc', 'fastball_bip_swung_perc', 
#                 'fastball_taken_strike_perc', 'fastball_est_woba', 'fastball_babip', 
#                 'fastball_iso_value', 'breaking_perc_faced', 'breaking_chase_perc', 
#                 'breaking_bip_swung_perc', 'breaking_taken_strike_perc', 
#                 'breaking_est_woba', 'breaking_babip', 'breaking_iso_value', 
#                 'offspeed_perc_faced', 'offspeed_chase_perc', 'offspeed_bip_swung_perc', 
#                 'offspeed_taken_strike_perc','offspeed_est_woba', 'offspeed_babip', 
#                 'offspeed_iso_value','overall_breaking_perc', 'count_cat_breaking_perc', 
#                 'overall_fastball_perc', 'count_cat_fastball_perc', 'overall_offspeed_perc', 
#                 'count_cat_offspeed_perc', 'pitchout_perc_faced', 'L5_fastball_perc', 
#                 'L15_fastball_perc', 'L5_breaking_perc', 'L15_breaking_perc',
#                 'L5_offspeed_perc', 'L15_offspeed_perc', 'L5_strike_perc', 
#                 'L15_strike_perc', 'PB_fastball', 'PB_breaking', 'PB_offspeed', 
#                 'sz_bot', 'sz_top']

# def final_model_preprocess(model_df):
#     df = model_df.copy()
    
#     #one hot encode certain columns
#     one_hot_ce = ce.OneHotEncoder(cols=one_hot_cols, drop_invariant=False, 
#                                   return_df=True, verbose=10, use_cat_names=True)
    
#     one_hot_df = df[one_hot_cols].copy() 
#     one_hot_df = one_hot_ce.fit_transform(one_hot_df)
    
#     #binary encode columns w/ lots of categories
#     binary_ce = ce.BinaryEncoder(cols=binary_cols, return_df=True)
    
#     binary_df = df[binary_cols].copy()
#     binary_df = binary_ce.fit_transform(binary_df)
    
#     #grab the remaining features
#     other_cols = set(df.columns) - set(one_hot_cols + binary_cols)
#     other_df = df[other_cols].copy()
    
#     #merge everything back together
#     df = pd.concat([one_hot_df, binary_df, other_df], axis=1)
#     return df

# model_df = final_model_preprocess(model_df)

In [414]:
model_df.shape

(6188, 91)

In [415]:
#Todo:

# - pickle the model_df and export

# - scale the data (probably w/ robust scaler)

# - split into train and test

# - for non tree based models: encode categorical variables

# - train models and evaluate predictions

In [416]:
vals = list(pitcher_id_dict.keys())
keys = list(pitcher_id_dict.values())
reverse_dict = dict(zip(keys, vals))

pickle_path = '/home/ec2-user/SageMaker/RC-v1.2-Predictive-Modelling/pitcher_df_pickles/'
pitcher_name = reverse_dict[pitcher_id].split()[1]

model_df.to_pickle(path=(pickle_path + pitcher_name + '_df.pkl'),compression='zip')